In [1]:
!pip install hnswlib 


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-2.2.0-cp311-cp311-win_amd64.whl (12.9 MB)
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp311-cp311-win_amd64.whl size=149905 sha256=f7d07f889c692f831645956894027eeb74ad155ea9d11b0f3d352e414e0eaf86
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\78\cf\dd\f68cc40d533e8203a7207eb44fbb32cf1243c9fea1962cfa23
Successfully built hnswlib



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install numpy scikit-learn matplotlib

     --------------------------------------- 11.1/11.1 MB 16.4 MB/s eta 0:00:00
     ---------------------------------------- 7.8/7.8 MB 12.8 MB/s eta 0:00:00
  Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl (44.8 MB)
     -------------------------------------- 301.8/301.8 kB 6.2 MB/s eta 0:00:00
     -------------------------------------- 219.8/219.8 kB 6.8 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ---------------------------------------- 2.2/2.2 MB 10.8 MB/s eta 0:00:00
     ---------------------------------------- 56.0/56.0 kB ? eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 10.9 MB/s eta 0:00:00
     -------------------------------------- 106.9/106.9 kB 6.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import hnswlib
import numpy as np
print("NumPy version:", np.__version__)

NumPy version: 2.2.0


In [8]:
!pip show hnswlib

Name: hnswlib
Version: 0.8.0
Summary: hnswlib
Home-page: https://github.com/yurymalkov/hnsw
Author: Yury Malkov and others
Author-email: 
License: 
Location: C:\Users\Pc\Downloads\hnsw benachmark py\hnsw_env\Lib\site-packages
Requires: numpy
Required-by: 


In [9]:

# Set parameters
dim = 128  # Dimension of the vectors

num_elements = 10000  # Number of elements to add
# Initialize the index
p = hnswlib.Index(space='l2', dim=dim)  # 'l2' refers to the Euclidean distance



In [10]:

# Generate random data

data = np.float32(np.random.random((num_elements, dim)))



In [11]:

# Set the number of threads used during the build process

p.set_num_threads(4)

# Build the index

p.init_index(max_elements=num_elements, ef_construction=200, M=16)

p.add_items(data)



In [12]:

# Set the exploration factor (ef)

p.set_ef(50)  # ef should always be greater than k



In [13]:

# Generate random queries

query_data = np.float32(np.random.random((100, dim)))

# Perform the query

labels, distances = p.knn_query(query_data, k=10)



In [14]:

assert len(labels) == 100  # Ensure we get results for all queries

assert labels.shape[1] == 10  # Ensure each query returns 10 nearest neighbors



In [15]:

# Compare results with brute-force search

from sklearn.metrics.pairwise import euclidean_distances

true_distances = euclidean_distances(query_data, data)

true_labels = np.argsort(true_distances, axis=1)[:, :10]

# Check if the HNSW results match the brute-force results

accuracy = np.mean([np.isin(labels[i], true_labels[i]).sum() for i in range(len(labels))])

print(f"Accuracy: {accuracy}")



Accuracy: 6.42


In [16]:

import time

start_time = time.time()

p.knn_query(query_data, k=10)

end_time = time.time()

print(f"Query time: {end_time - start_time} seconds")



Query time: 0.031002044677734375 seconds


In [ ]:
# ef>k takes more time
#with cosine
# Yes, in this code the filter function (filter_function(idx)) takes an ID and returns True/False based on whether the point's label matches the target label, by checking self.labels[idx] == target_label.

# Data: (N=3000) × (d=16) matrix of uniform random values
# Labels: N-length array of categorical values ['a','b','c']
# IDs: Array [0 to N-1] matching data indices
# Queries: (num_queries=100) × (d=16) matrix of uniform random values

# The filtering is categorical, using the 1D label array to filter the d-dimensional data points during search.



# recall = |filtered_results ∩ true_results| / |true_results|

# Here's a simple example of the data format for N=5 points with d=4 dimensions:
# Data matrix (5×4):
# pythonCopydata = [
#     [0.1, 0.2, 0.3, 0.4],  # point 0
#     [0.5, 0.6, 0.7, 0.8],  # point 1
#     [0.2, 0.3, 0.4, 0.5],  # point 2
#     [0.6, 0.7, 0.8, 0.9],  # point 3
#     [0.3, 0.4, 0.5, 0.6]   # point 4
# ]
# Labels (5×1):
# pythonCopylabels = ['a', 'b', 'a', 'c', 'b']
# Query example (1×4):
# query = [0.4, 0.5, 0.6, 0.7]

# Yes, the numbers in the data matrix are embeddings - vectors representing the features/characteristics of each point in a d-dimensional space. In this example they're randomly generated but in real applications they would be meaningful embeddings like image features, text embeddings, etc.


# In this code, queries don't have labels - they're just feature vectors. The search finds the k nearest data points to each query point, but only among data points that have the specified label in the filter. So if filtering for label 'a', it will return the closest points that have label 'a', ignoring points with other labels


# Specificity = (Number of points passing filter) / (Total number of points)
# Recall = |Retrieved True Nearest Neighbors ∩ Actual True Nearest Neighbors| / |Actual True Nearest Neighbors|
# Latency Overhead = Filtered Query Time / Unfiltered Query Time


# ef_construction (higher = better recall, slower build)
# ef (higher = better recall, slower search)
# M (more connections = better recall, more memory)



# Yes, this makes sense:
# When 'a' is 33.3% of points:

# Filtered latency: 0.20ms
# Must search wider to find k neighbors

# When 'a' is 60% of points:

# Filtered latency: 0.11ms (faster)
# Can find k neighbors more quickly since more points qualify

# The latency decreased from 0.20ms to 0.11ms when specificity increased from 33.3% to 60% because:

# More qualifying points are found earlier in the search
# Less "wasted" exploration of points that don't match the filter
# Can search a smaller radius to find k valid neighbors

# So higher specificity = faster search times, which is exactly what we see here


# For recall: You can use index.getDataRetrieval() in HNSWLIB to get actual nearest neighbors, 



# When you do hnsw_index.knn_query(..., filter=filter_function), the filter is checking the data points' IDs, not anything about the queries 



In [ ]:
# Yes, tunable aspects are satisfied through:

# Adjustable EF values (ef_values)
# Tunable alpha in power law (skewed_data1)
# Tunable cluster_std in correlated data
# Adjustable range in range filter
# Configurable dim and num_elements in init

# All key parameters can be modified to test different scenarios.


# Yes, accuracy is measured through recall@k, which compares filtered results against unfiltered (ground truth) results. No need for separate accuracy metric.


# Filter friction is measured indirectly through:

# Latency differences (filtered vs unfiltered)
# Throughput impact
# Recall degradation
# Specificity percentages

# Together these show how the index "behaves" under different filter conditions